In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from model_constructor import ModelConstructor

import data_loading
import data_processing
import model_training
import data_visualisation

In [ ]:
all_data = data_loading.load_gestures_grouped_per_candidate(use_left_hand=True, use_right_hand=True)
all_candidates = list(all_data.keys())
data, labels = data_loading.get_data_and_labels_from_candidates(all_candidates, all_data, input_shape=(100, 3, 1))
i = 150
data_visualisation.plot_data_as_graph(data[i], title=f"{labels[i]}")
data_visualisation.plot_data_as_image(data[i], label=f"{labels[i]}")

# Testing reshaping

## The sample

In [ ]:
sample_data = [[540, 496, 543],
[531, 482, 541],
[523, 473, 538],
[512, 459, 536],
[502, 440, 533],
[486, 366, 528],
[468, 308, 524],
[448, 271, 522],
[426, 239, 513],
[401, 216, 507],
[403, 217, 506],
[358, 203, 498],
[282, 194, 484],
[238, 188, 468],
[217, 186, 442],
[205, 182, 408],
[198, 180, 317],
[192, 178, 252],
[188, 176, 224],
[186, 176, 205],
[185, 171, 199],
[180, 173, 191],
[180, 172, 185],
[179, 171, 183],
[176, 167, 182],
[176, 170, 178],
[176, 167, 177],
[174, 169, 176],
[174, 165, 175],
[174, 166, 173],
[173, 166, 173],
[173, 163, 171],
[170, 166, 170],
[170, 166, 169],
[169, 166, 168],
[170, 162, 167],
[168, 164, 166],
[169, 161, 167],
[167, 163, 164],
[167, 163, 163],
[166, 162, 163],
[167, 159, 179],
[168, 162, 217],
[166, 161, 302],
[165, 160, 373],
[165, 160, 420],
[178, 160, 451],
[278, 174, 477],
[362, 218, 496],
[417, 293, 507],
[457, 392, 515],
[486, 441, 522],
[508, 472, 528],
[525, 495, 533],
[538, 508, 537],
[549, 523, 540],
[557, 535, 544],
[564, 546, 546],
[569, 550, 547],
[575, 559, 550],
[577, 563, 550],
[580, 568, 551],
[582, 569, 556],
[586, 574, 556],
[588, 575, 555],
[589, 577, 556],
[590, 581, 556],
[591, 583, 558],
[593, 583, 559],
[592, 587, 558],
[593, 586, 560],
[593, 587, 560],
[593, 588, 560],
[594, 588, 560],
[596, 590, 559],
[595, 590, 560],
[595, 587, 561],
[597, 591, 560],
[596, 590, 560],
[597, 593, 561],
[596, 591, 561],
[597, 592, 562],
[596, 590, 562],
[597, 589, 562],
[597, 592, 562],
[597, 591, 561],
[596, 591, 561],
[596, 589, 563],
[596, 592, 562],
[598, 593, 563],
[597, 592, 560],
[598, 590, 561],
[596, 591, 560],
[598, 593, 562],
[596, 589, 563],
[598, 593, 563],
[595, 590, 562],
[597, 590, 561],
[597, 589, 562],
[597, 592, 562]
]

## Python workflow

In [ ]:
plt.plot(sample_data)
print(len(sample_data))

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_model.tflite")

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

signature = interpreter.get_signature_runner()
array = np.array(data_processing.preprocess_data(sample_data))
array = np.reshape(array, (20, 5, 3))
# with np.printoptions(threshold=np.inf, precision=2, suppress=True):
#     print(f"array (shape={array.shape}) with reshaping through np: ")
#     print(array)

#     print(f"reshaped (shape={reshaped.shape}) with manual reshaping: ")
#     print(reshaped)

# np.testing.assert_array_almost_equal(np.ndarray.flatten(array), np.ndarray.flatten(reshaped))

# This is the output we want
with np.printoptions(threshold=np.inf, precision=2, suppress=True):
    print("array:\n", np.ravel(array))
    # print("array:\n", array)


array = np.expand_dims(array, axis=0)
output = signature(sensor_image=array.astype(np.float32))
print(output)

## C++ (manual) workflow

In [ ]:
after_processing = data_processing.preprocess_data(sample_data) # simulate data processing
# after_processing = sample_data
after_processing = np.array(after_processing).astype(np.float32)

# On the device we store the data transposed
transposed = after_processing.transpose()
# transposed = after_processing

# Print transposed data
with np.printoptions(threshold=np.inf, precision=2, suppress=True):
    print("Transposed data:", transposed)


In [ ]:
# After transposing we 'cast' the data to the shape of the input tensor
# In C++ this is done by casting the data to a float pointer with the three dimensions
DIM1 = 20
DIM2 = 5
DIM3 = 3

cast_array = np.reshape(transposed, (DIM3, DIM2, DIM1))

reshaped = [0] * DIM1 * DIM2 * DIM3
index = 0
for j in range(DIM2):
    for i in range(DIM1):
        for k in range(DIM3):
            reshaped[index] = cast_array[k, j, i]
            index += 1

reshaped = np.array(reshaped)
print(reshaped.shape)
with np.printoptions(threshold=np.inf, precision=2, suppress=True):
    print("Reshaped data:\n", reshaped)

In [ ]:
# Check whether they are equal

np.testing.assert_array_almost_equal(np.ndarray.flatten(array), np.ndarray.flatten(reshaped))